In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [15]:
# read all tweets
df = pd.read_csv('./dataset_filter_240109.csv', index_col=0, nrows=30000)

In [16]:
df.head()

,tweet_id,author_id,created_at,retweet_count,reply_count,like_count,quote_count,en_text,stanza_output,stanza_named_entities,...,purity,non-moral,mdate,ordinal_mdate,stance_pro_rus,stance_pro_ukr,stance_unsure,sentiment_negative,sentiment_neutral,sentiment_positive
0,1540451336876986373,15432891,2022-06-24,0,0.0,0,0,Weekend selection: Zelensky was not prepared f...,"[[{'id': 1, 'text': 'Weekend', 'upos': 'NOUN',...","[{'text': 'Zelensky', 'type': 'PERSON', 'start...",...,0.072048,0.617735,2022-06,4,0,0,1,0,1,0
1,1516696238212661253,78567974,2022-04-20,1,1.0,0,0,"The Ukrainian war, Charles Michel of Kiev, the...","[[{'id': 1, 'text': 'The', 'upos': 'DET', 'xpo...","[{'text': 'The Ukrainian war', 'type': 'EVENT'...",...,0.070797,0.606203,2022-04,2,0,0,1,0,1,0
2,1587766197298548738,145535271,2022-11-02,0,0.0,2,0,The invention of the Shark drone is a new chap...,"[[{'id': 1, 'text': 'The', 'upos': 'DET', 'xpo...","[{'text': 'Shark', 'type': 'PRODUCT', 'start_c...",...,0.015667,0.614123,2022-11,9,0,0,1,0,1,0
3,1586193152175529984,145535271,2022-10-29,0,0.0,5,0,Will nuclear war be over? Those countries that...,"[[{'id': 1, 'text': 'Will', 'upos': 'AUX', 'xp...",[],...,0.071667,0.614273,2022-10,8,0,0,1,1,0,0
4,1580452730212331520,145535271,2022-10-13,0,0.0,3,0,Boys fight: Students fight in coaching center....,"[[{'id': 1, 'text': 'Boys', 'upos': 'NOUN', 'x...","[{'text': 'Russia', 'type': 'GPE', 'start_char...",...,0.242989,0.621450,2022-10,8,0,1,0,1,0,0


In [17]:
df.columns

Index(['tweet_id', 'author_id', 'created_at', 'retweet_count', 'reply_count',
       'like_count', 'quote_count', 'en_text', 'stanza_output',
       'stanza_named_entities', 'stance', 'channel', 'country', 'verified',
       'follower_count', 'sentiment.positive', 'sentiment.neutral',
       'sentiment.negative', 'with_media', 'anger_num', 'anticipation_num',
       'disgust_num', 'fear_num', 'joy_num', 'negative_num', 'positive_num',
       'sadness_num', 'surprise_num', 'trust_num', 'stopword_num', 'word_num',
       'sentence_num', 'care', 'fairness', 'loyalty', 'authority', 'purity',
       'non-moral', 'mdate', 'ordinal_mdate', 'stance_pro_rus',
       'stance_pro_ukr', 'stance_unsure', 'sentiment_negative',
       'sentiment_neutral', 'sentiment_positive'],
      dtype='object')

In [18]:
len(df)

30000

In [19]:
import string
string.digits

'0123456789'

In [20]:
'‘’”“'

'‘’”“'

In [21]:
punctuations = string.punctuation+'‘’”“'

In [27]:
from gensim import corpora, models
from gensim.models import CoherenceModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

# 下载停用词
import nltk
nltk.download('stopwords')

# 初始化词干化器
stemmer = PorterStemmer()

# 初始化停用词
stop_words = set(stopwords.words('english'))
stop_words.update(['re', 'http', '\'s', 'rt'])

# 定义文本预处理函数
def preprocess_text(text):
    # 分词
    tokens = word_tokenize(text)
    # 去除标点符号
    tokens = [token for token in tokens if token not in punctuations]
    # 小写化
    tokens = [token.lower() for token in tokens]
    # 词干化
    tokens = [stemmer.stem(token) for token in tokens]
    # 去停用词
    tokens = [token for token in tokens if token not in stop_words]
    # 去数字
    tokens = [token for token in tokens if token not in string.digits]
   
    return tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fanhongrui/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
# 对文本进行预处理
df['en_text_tokens'] = df['en_text'].apply(preprocess_text)

# 创建词典
dictionary = corpora.Dictionary(df['en_text_tokens'])

# 创建文档-词频矩阵
corpus = [dictionary.doc2bow(text) for text in df['en_text_tokens']]

In [29]:
# 初始化 coherence 和最优的主题数量
best_coherence = -1
best_topic_num = 0

# 尝试不同的主题数量
for num_topics in range(2, 7):  # 假设尝试主题数量从2到6
    # 训练LDA模型
    lda_model = models.LdaModel(corpus, id2word=dictionary, num_topics=num_topics)
    
    # 计算主题一致性
    coherence_model = CoherenceModel(model=lda_model, texts=df['en_text_tokens'], dictionary=dictionary, coherence='c_v')
    coherence = coherence_model.get_coherence()
    
    # 如果当前主题数量的一致性比之前的更好，则更新最优值
    if coherence > best_coherence:
        best_coherence = coherence
        best_topic_num = num_topics

# 输出最佳的主题数量和对应的一致性
print(f"Best number of topics: {best_topic_num}")
print(f"Coherence score: {best_coherence}")

# 输出最佳主题最具代表性的15个词语
for topic_id in range(best_topic_num):
    print(f"Topic {topic_id}:")
    topic_terms = lda_model.show_topic(topic_id, topn=15)
    for term, prob in topic_terms:
        print(f"{term}: {prob}")
    print()

Best number of topics: 4
Coherence score: 0.28195436532433416
Topic 0:
russian: 0.06476058065891266
ukrainian: 0.03469914570450783
war: 0.03445715457201004
ukrain: 0.03306416794657707
toward: 0.031312331557273865
kiev: 0.020745592191815376
troop: 0.017118265852332115
russia: 0.012981101870536804
militari: 0.012879774905741215
warn: 0.011613159440457821
advanc: 0.01052046287804842
refuge: 0.010152238421142101
citi: 0.009818200021982193
forc: 0.009746570140123367
attack: 0.009545856155455112

Topic 1:
war: 0.05950278416275978
ukrain: 0.05217037349939346
russian: 0.03235456719994545
``: 0.029044726863503456
crime: 0.028922926634550095
'': 0.024890685454010963
warn: 0.023673128336668015
russia: 0.022666119039058685
ukrainian: 0.01302951481193304
moscow: 0.011965116485953331
commit: 0.011680434457957745
presid: 0.011335963383316994
...: 0.008660303428769112
mariupol: 0.008502935990691185
putin: 0.007906910963356495

Topic 2:
war: 0.0927477553486824
ukrain: 0.05976678058505058
russia: 0.0283

In [11]:
# # 初始化一个空列表来存储每个文档的最大主题
# max_topic_list = []

# # 对每个文档进行转换，得到主题分布
# topic_distributions = [lda_model.get_document_topics(doc) for doc in corpus]

# # 遍历每个文档，找到概率最大的主题并添加到 max_topic_list 中
# for doc_topics in topic_distributions:
#     max_prob = 0.0
#     max_topic_id = -1
#     for topic_id, prob in doc_topics:
#         if prob > max_prob:
#             max_prob = prob
#             max_topic_id = topic_id
#     max_topic_list.append(max_topic_id)

# # 将最大主题列表添加到 DataFrame 的新列 'LDA_topic' 中
# df['LDA_topic'] = max_topic_list


In [12]:
# # 假设 lda_model 是你训练好的 LDA 模型

# # 保存模型到文件
# lda_model.save('lda_model')

# # 若要加载模型，可以使用以下代码
# # 加载模型
# # loaded_lda_model = models.LdaModel.load('lda_model')


In [30]:
import pyLDAvis
import pyLDAvis.gensim_models

lda_model = models.LdaModel(corpus, id2word=dictionary, num_topics=best_topic_num)
lda_vis_data = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)

# 显示可视化结果
pyLDAvis.display(lda_vis_data)